In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def gaussian_filter(support_size, sigma):

    center = support_size // 2
    gf = np.zeros(support_size)

    for i in range(support_size):
        x = i - center
        gf[i] = 1.0 / (np.sqrt(2 * np.pi) * sigma) * np.exp(-(x**2) / (2 * sigma**2))
        
    gf = gf / np.sum(gf)

    return gf

In [ ]:
def gaussian_derivative(support_size, sigma):
    center = support_size // 2
    # Initialize derivative of Gaussian (DoG)
    dog = np.zeros(support_size)

    for i in range(support_size):
        x = i - center
        dog[i] = -x / (np.sqrt(2 * np.pi) * sigma**3) * np.exp(-(x**2) / (2 * sigma**2))
        
    return dog

In [ ]:
# Example usage:

filter_size = 5

sigma = 0.9

gf = gaussian_filter(filter_size, sigma) 
dog = gaussian_derivative(filter_size, sigma)

print(gf, np.sum(gf))
print(dog, np.sum(dog))

In [ ]:
def gaussian_derivative_2D(sigma, size, axis):
    """
    Computes the derivative of a Gaussian filter.

    Parameters:
    - sigma: Standard deviation of the Gaussian filter.
    - size: Support size of the filter (size of the filter window).
    - axis: The axis for the derivative ('x' or 'y').

    Returns:
    - The derivative of the Gaussian filter as a 2D array.
    """
    # Create a grid of (x, y) values
    half_size = size // 2
    x = np.arange(-half_size, half_size + 1)
    y = np.arange(-half_size, half_size + 1)
    xx, yy = np.meshgrid(x, y)

    # Compute the Gaussian function
    gaussian = np.exp(-(xx**2 + yy**2) / (2 * sigma**2))

    # Compute the derivative based on the axis
    if axis == 'x':
        derivative = -xx / (sigma**2) * gaussian
    elif axis == 'y':
        derivative = -yy / (sigma**2) * gaussian
    else:
        raise ValueError("Invalid axis. Choose 'x' or 'y'.")

    return derivative


def visualize_filter(filter_array, title):
    """
    Visualizes the 2D filter as an image.

    Parameters:
    - filter_array: 2D array of the filter.
    - title: Title of the plot.
    """
    plt.imshow(filter_array, cmap='gray', extent=[-1, 1, -1, 1])
    plt.colorbar()
    plt.title(title)
    plt.axis('off')
    plt.show()


In [ ]:
# Example Usage
sigma = 1.0       # Standard deviation of the Gaussian
size = 7          # Support size of the filter (must be an odd number)

# Compute derivative of Gaussian filter
dx_filter = gaussian_derivative_2D(sigma, size, 'x')
dy_filter = gaussian_derivative_2D(sigma, size, 'y')

# Visualize filters
visualize_filter(dx_filter, "Derivative of Gaussian (dx)")
visualize_filter(dy_filter, "Derivative of Gaussian (dy)")

In [ ]:

def non_maximum_suppression(gradient_magnitude, gradient_direction):
    """
    Perform Non-Maximum Suppression on the gradient magnitude image.
    """
    rows, cols = gradient_magnitude.shape
    nms = np.zeros((rows, cols), dtype=np.float64)

    angle = gradient_direction % 180  # Normalize angles to [0, 180)

    for i in range(1, rows - 1):
        for j in range(1, cols - 1):
            # Determine the two neighbors to compare with
            if (0 <= angle[i, j] < 22.5) or (157.5 <= angle[i, j] <= 180):
                neighbors = [gradient_magnitude[i, j - 1], gradient_magnitude[i, j + 1]]
            elif 22.5 <= angle[i, j] < 67.5:
                neighbors = [gradient_magnitude[i - 1, j + 1], gradient_magnitude[i + 1, j - 1]]
            elif 67.5 <= angle[i, j] < 112.5:
                neighbors = [gradient_magnitude[i - 1, j], gradient_magnitude[i + 1, j]]
            else:
                neighbors = [gradient_magnitude[i - 1, j - 1], gradient_magnitude[i + 1, j + 1]]

            # Suppress non-maximum pixels
            if gradient_magnitude[i, j] >= max(neighbors):
                nms[i, j] = gradient_magnitude[i, j]

    return nms


def hysteresis_thresholding(image, low_threshold, high_threshold):
    """
    Perform hysteresis thresholding on the image.
    """
    rows, cols = image.shape
    result = np.zeros((rows, cols), dtype=np.uint8)

    strong = 255
    weak = 50

    # Define strong and weak edges
    strong_i, strong_j = np.where(image >= high_threshold)
    weak_i, weak_j = np.where((image >= low_threshold) & (image < high_threshold))

    result[strong_i, strong_j] = strong
    result[weak_i, weak_j] = weak

    # Perform edge tracking by hysteresis
    for i in range(1, rows - 1):
        for j in range(1, cols - 1):
            if result[i, j] == weak:
                if any(result[i + di, j + dj] == strong for di in [-1, 0, 1] for dj in [-1, 0, 1]):
                    result[i, j] = strong
                else:
                    result[i, j] = 0

    return result

In [ ]:
# Load the image in grayscale
filename = "GFP_06-DAPI.tif"
image = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)

if image is None:
    print(f"Error: Could not load image '{filename}'. Please check the file path.")

# Step 1: Effect of Convolution with Derivative of Gaussian
# Apply GaussianBlur (smoothing)
blurred = cv2.GaussianBlur(image, (5, 5), 1.4)

# Compute gradients (Sobel derivative of Gaussian)
grad_x = cv2.Sobel(blurred, cv2.CV_64F, 1, 0, ksize=3)
grad_y = cv2.Sobel(blurred, cv2.CV_64F, 0, 1, ksize=3)

# Gradient magnitude and direction
gradient_magnitude = np.sqrt(grad_x**2 + grad_y**2)
gradient_direction = np.arctan2(grad_y, grad_x) * (180 / np.pi)  # Convert to degrees

# Step 2: Effect of Non-Maximum Suppression
nms_image = non_maximum_suppression(gradient_magnitude, gradient_direction)

# Step 3: Effect of Hysteresis Thresholding
low_threshold = 50
high_threshold = 100
hysteresis_image = hysteresis_thresholding(nms_image, low_threshold, high_threshold)

# Display results
#plt.figure(figsize=(12, 8))

#plt.subplot(2, 3, 1)
plt.imshow(image, cmap='gray')
plt.title("Original Image")
plt.axis('off')
plt.show()

#plt.subplot(2, 3, 2)
plt.imshow(blurred, cmap='gray')
plt.title("Blurred (Gaussian)")
plt.axis('off')
plt.show()

#plt.subplot(2, 3, 3)
invert_gm = gradient_magnitude * (-1.0) + gradient_magnitude.max()
plt.imshow(gradient_magnitude.max() - gradient_magnitude, cmap='gray')
plt.title("Gradient Magnitude")
plt.axis('off')
plt.show()


plt.imshow(grad_x, cmap='gray')
plt.title("Horizontal Gradient")
plt.axis('off')
plt.show()

plt.imshow(grad_y, cmap='gray')
plt.title("Vertical Gradient")
plt.axis('off')
plt.show()

#plt.subplot(2, 3, 4)
plt.imshow(nms_image.max() - nms_image, cmap='gray')
plt.title("Non-Maximum Suppression")
plt.axis('off')
plt.show()

#plt.subplot(2, 3, 5)
plt.imshow(hysteresis_image.max() - hysteresis_image, cmap='gray')
plt.title("Hysteresis Thresholding")
plt.axis('off')

#plt.tight_layout()
plt.show()